In [2]:
import pandas as pd
import os
import re
import numpy as np

## 1. Importar la información

In [115]:
data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data"
emicron_19 = pd.read_csv(os.path.join(data_path,"EMICRON_19.csv"),header = 0)
emicron_21 = pd.read_csv(os.path.join(data_path,"EMICRON_21.csv"),header = 0)
emicron_22 = pd.read_csv(os.path.join(data_path,"EMICRON_22.csv"),header = 0)

C:\Users\carlo\AppData\Local\Temp\ipykernel_17808\33973033.py:2: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  emicron_19 = pd.read_csv(os.path.join(data_path,"EMICRON_19.csv"),header = 0)


In [116]:
emicron_19['F_EXP'] = emicron_19['F_EXP'].apply(lambda x: float(x.replace(',', '.')))

In [117]:
emicron = pd.concat([emicron_19, emicron_21, emicron_22])

In [120]:
emicron['F_EXP'][emicron['OLA']==2022]

0         25.099238
1         22.117173
2         34.341336
3         21.413039
4        122.952097
            ...    
84748     15.316276
84749     21.269300
84750     10.848121
84751     11.055347
84752     11.244637
Name: F_EXP, Length: 84753, dtype: float64

## 2. Filtrado de la base de datos
Se procederá a filtrar la base de datos con base en unos elementos particulares:  
1. Actividades económicas que cubre EMICRON y que no serán tenidas en cuenta en CENU.  
2. Ubicación del micronegocio. Ya que se entrenará un modelo para puestos fijos y micronegocios en vivienda, y otro para puestos móviles. Con micronegocios o puestos móviles que sean visibles.   
3. Sólo serán informativos aquellos micronegocios que hayan operado durante el mes anterior a la aplicación de la encuesta a partir de valores en ventas del mes pasado que sean diferentes de cero (como no existen valores menores que cero, entonces los que son mayores a cero).
4. Otras variables como el número de establecimientos, que permitan mantener una información homogénea.

### 2.1 Actividades económicas que cubrirá el CENU 
Emicron incluye preguntas a los sectores de agricultura, ganadería, caza, silvicultura y pesca; además, de minería. Debido a que estos sectores no harán parte del Universo del Censo Económico, se descartarán de la base de datos:

In [121]:
emicron = emicron[~(emicron['GRUPOS12'].isin([1,2,13]))]
emicron.shape

(214010, 210)

In [122]:
pd.crosstab(emicron['GRUPOS12'],emicron['GRUPOS4'])

GRUPOS4,2,3,4
GRUPOS12,,,
3,27522,0,0
4,0,0,16889
5,0,72734,0
6,0,0,27319
7,0,0,23917
8,0,0,2310
9,0,0,12428
10,0,0,2424
11,0,0,2181


In [123]:
emicron[emicron['OLA']==2022].shape

(73966, 210)

### 2.2 Ubicación del micronegocio
Para el entrenamiento de los modelos se utilizará la ubicación de micronegocios en 1: Vivienda, 2: Local (puestos fijos), 4: Ambulante, por lo tanto sólo estas categorías se mantendrán. De igual manera, el CENU, sólo recolectará información de actividades económicas visibles, por lo tanto se filtrará por este tipo de micronegocios.

In [124]:
print(emicron['P3053'].value_counts())

1    73214
3    43516
2    33583
5    26785
4    26738
6     7928
8     1146
7     1100
Name: P3053, dtype: int64


In [125]:
pd.crosstab(emicron['P3053'],emicron['P469'])

P469,1.0,2.0
P3053,,
1,38263,34951
2,31273,2310
4,9524,820
5,21958,4827
7,374,726


In [126]:
emicron['P3053'][emicron['P469']==2].value_counts()

1    34951
5     4827
2     2310
4      820
7      726
Name: P3053, dtype: int64

In [127]:
emicron.shape

(214010, 210)

In [128]:
print(emicron['P3053'].value_counts())

# Seleccionar sólo viviendas, establecimientos y ambulantes
emicron = emicron[emicron['P3053'].isin([1,2,4])]

# Reemplazar los códigos en función de la codificación que se recibirá en el CENU
# 1: Establecimiento fijo
# 3: Vivienda con actividad económica visible
def rule(i):
    if i == 1:
        return 3
    elif i == 2:
        return 1
    return i

emicron['P3053'] = list(map(rule,emicron['P3053']))
print(emicron['P3053'].value_counts())


1    73214
3    43516
2    33583
5    26785
4    26738
6     7928
8     1146
7     1100
Name: P3053, dtype: int64
3    73214
1    33583
4    26738
Name: P3053, dtype: int64


In [129]:
emicron[emicron['OLA']==2022].shape

(44129, 210)

In [130]:
# Visibilidad del micronegocio
emicron = emicron[(emicron['P469'] == 1)]
emicron.shape

(79060, 210)

In [131]:
emicron[emicron['OLA']==2022].shape

(26590, 210)

### 2.3 Ventas del mes anterior
Eliminar las filas cuyas ventas del año pasado son missing y cero

In [132]:
# Valores nulos ventas
print("Valores nulos ventas: ", emicron["VENTAS_MES_ANTERIOR"].isna().sum())

# Valores cero ventas
print("Ventas en cero: ", (emicron["VENTAS_MES_ANTERIOR"]==0).sum())

# Eliminar los registros con venta cero
emicron = emicron[~(emicron["VENTAS_MES_ANTERIOR"] == 0)]

Valores nulos ventas:  0
Ventas en cero:  2309


In [133]:
emicron.shape

(76751, 210)

In [134]:
emicron[emicron['OLA']==2022].shape

(25955, 210)

### 2.4 Número de establecimientos
Eliminar las filas cuyo número de estabecimientos es diferente a 1, ya que la información que recolecta el CENU es por establecimiento y la recolectada por EMICRON es a nivel de negocio, pero a veces se presenta información también solo por establecimiento cuando el negocio tiene más de un establecimiento. Para evitar eso, sólo utilizar los micronegocios que reportan un sólo establecimiento comercial, que representan el 99.36 % de la muestra (casi todos).

In [135]:
emicron['P3054'] = pd.to_numeric(emicron['P3054'], errors='coerce')
emicron['P3054'].value_counts()

1.0      76242
2.0        322
99.0       126
3.0         37
4.0          9
5.0          7
6.0          4
999.0        1
10.0         1
7.0          1
8.0          1
Name: P3054, dtype: int64

In [136]:
emicron['P3054'][emicron['OLA']==2022].value_counts() / (emicron['P3054'][emicron['OLA']==2022].value_counts().sum())

1.0     0.995492
2.0     0.003622
99.0    0.000424
3.0     0.000385
4.0     0.000077
Name: P3054, dtype: float64

In [137]:
emicron = emicron[(emicron['P3054'] == 1)]
emicron.shape

(76242, 210)

In [138]:
emicron[emicron['OLA']==2022].shape

(25838, 210)

In [139]:
emicron['P3053'].value_counts()

3    36746
1    30325
4     9171
Name: P3053, dtype: int64

In [140]:
emicron['P3053'][emicron['OLA']==2022].value_counts()

3    12978
1     9804
4     3056
Name: P3053, dtype: int64

## 3. Construcción y transformación de variables
1. Se creará una agrupación para la variable del número total de empleados  
2. Se agregarán los costos y los gastos en una sola variable (CYG)  
3. AREA se recodificará para que muestre si la observación pertenece a una ciudad capital del departamento  
4. Contabilidad

### 3.1 Personal Ocupado

In [141]:
emicron['P3032_1'] = pd.to_numeric(emicron['P3032_1'], errors='coerce')
emicron['P3032_2'] = pd.to_numeric(emicron['P3032_2'], errors='coerce')
emicron['P3032_3'] = pd.to_numeric(emicron['P3032_3'], errors='coerce')

emicron['POTTOT'] = emicron[['P3032_1','P3032_2','P3032_3']].sum(axis=1)

### 3.2 Costos y Gastos agregados

In [142]:
emicron['CYG'] = emicron[['COSTOS_MES_ANTERIOR','GASTOS_MES']].sum(axis=1)

### 3.3 Variable de Área
Recodificar la variable de Área en función de si es una ciudad principal o un área metropolitana dentro del departamento

In [143]:
emicron['AREA'] = pd.to_numeric(emicron['AREA'], errors='coerce')

def rule_city(i):
    if pd.isna(i) == True:
        return 0
    else:
        return 1
    return i

emicron['CPAM'] = list(map(rule_city,emicron['AREA']))

### 3.4 Variables de formalidad

In [144]:
## Formalidad del micronegocio
# Tiene RUT
emicron['P1633'] = [0 if num == 2 else num for num in emicron['P1633']]
# Está inscrito en CC
emicron['P1055'] = [0 if num == 2 else num for num in emicron['P1055']]

### 3.5 Forma de llevar la contabilidad

In [145]:
## Manera de llevar contabilidad
# Se unifica 4: informes financieros en 1: Balance General P&G
print(emicron['P640'].value_counts())
emicron['P640'] = [1 if num == 4 else num for num in emicron['P640']]
print(emicron['P640'].value_counts())

# Se cambia el 5 por 4, para homogenizar con los códigos de CENU
emicron['P640'] = [4 if num == 5 else num for num in emicron['P640']]
print(emicron['P640'].value_counts())

5    37254
3    29135
2     6652
1     2815
4      386
Name: P640, dtype: int64
5    37254
3    29135
2     6652
1     3201
Name: P640, dtype: int64
4    37254
3    29135
2     6652
1     3201
Name: P640, dtype: int64


## 4. Renombrar variables

In [147]:
nombre_estandar = {'P3053': 'UBICA',
                   'P1633': 'IDRUT',
                   'P640': 'REG_CONT',
                   'P1055':'CAMCOMER',
                   'P639' : 'ANOS_ESPACIO'
               }
emicron.rename(columns=nombre_estandar, inplace=True)

## 5. Selección de variables

In [148]:
var_estab = ['id','VENTAS_MES_ANTERIOR','POTTOT','REMUNERACION_TOTAL','CYG','COD_DEPTO','CPAM','GRUPOS4','UBICA','IDRUT','REG_CONT','CAMCOMER','OLA','F_EXP'] 
var_mov = ['id','VENTAS_MES_ANTERIOR','POTTOT','REMUNERACION_TOTAL','CYG','COD_DEPTO','CPAM','GRUPOS4','ANOS_ESPACIO','OLA','F_EXP']

In [149]:
emicron['UBICA'].value_counts()

3    36746
1    30325
4     9171
Name: UBICA, dtype: int64

In [150]:
emicron_estab = emicron[emicron['UBICA'].isin([1,3])]
emicron_estab = emicron_estab[var_estab]
emicron_estab.to_csv(os.path.join(data_path,"emicron_establecimientos.csv"), index = False)

emicron_mov = emicron[emicron['UBICA']==4]
emicron_mov = emicron_mov[var_mov]
emicron_mov.to_csv(os.path.join(data_path,"emicron_moviles.csv"), index = False)

In [157]:
emicron['F_EXP'][emicron['OLA']==2022].describe()

count    25838.000000
mean        54.076358
std         74.720172
min          1.173286
25%         12.890374
50%         25.169194
75%         67.063328
max       1020.714919
Name: F_EXP, dtype: float64

In [158]:
emicron_estab[emicron_estab['F_EXP']>1020]

,id,VENTAS_MES_ANTERIOR,POTTOT,REMUNERACION_TOTAL,CYG,COD_DEPTO,CPAM,GRUPOS4,UBICA,IDRUT,REG_CONT,CAMCOMER,OLA,F_EXP
9979,481690813,1155914,0.0,0.0,868364,5,0,3,1,0,4,0,2019,1186.211492
3680,535690611,2528695,0.0,0.0,1373652,5,0,3,1,0,4,0,2021,1036.917748
11555,538749012,1700000,1.0,85000.0,1315000,25,0,4,3,0,3,0,2021,1170.001229
11560,538750111,800000,1.0,0.0,690000,25,0,3,1,1,3,1,2021,1039.279416
11602,538760511,4500000,2.0,2500000.0,3665000,25,0,3,1,1,3,1,2021,1182.400781
23089,543310812,550000,0.0,0.0,462000,76,0,3,3,0,3,0,2021,1111.160063
36926,548803912,10000,0.0,0.0,5000,23,0,2,3,0,4,0,2021,1095.166446
42946,551233511,2250000,0.0,0.0,1925000,5,0,3,1,0,4,0,2021,1024.258806
44091,551571211,750000,1.0,42000.0,463000,25,0,4,1,1,3,1,2021,1042.318563
44092,551571512,11000000,5.0,1130000.0,4190000,25,0,3,3,1,3,0,2021,1143.044133


In [154]:
emicron['F_EXP'][emicron['OLA']==2019]

2        423.131668
10        81.697983
14        95.833409
16       123.030932
17       105.952890
            ...    
86951     43.479839
86954    257.214252
86955     70.608422
86956     13.695497
86960     18.448384
Name: F_EXP, Length: 28067, dtype: float64

In [187]:
#emicron = emicron[pd.unique(var_estab + var_mov)]

## 6. One-Hot Encoding

In [114]:
from sklearn.preprocessing import OneHotEncoder

### 6.1 Creación variables dummies

In [188]:
ohe_dpto = OneHotEncoder().fit(emicron[['COD_DEPTO']])
drop_enc = ohe_dpto.transform(emicron[['COD_DEPTO']]).toarray()
drop_enc_df = pd.DataFrame(drop_enc,columns = ohe_dpto.get_feature_names_out())
drop_enc_df
drop_enc_df.drop('COD_DEPTO_5', axis=1, inplace = True) # Utilizar como categoría de referencia 5: Antioquia

emicron = emicron.join(drop_enc_df)

ohe_ubi = OneHotEncoder().fit(emicron[['UBICA']])
drop_enc = ohe_ubi.transform(emicron[['UBICA']]).toarray()
drop_enc_df = pd.DataFrame(drop_enc,columns = ohe_ubi.get_feature_names_out())
drop_enc_df.drop('UBICA_3', axis=1, inplace = True)  # Utilizar como categoría de referencia 3: Vivienda (aplica sólo para Establecimientos)

emicron = emicron.join(drop_enc_df)

ohe_sector = OneHotEncoder().fit(emicron[['GRUPOS4']])
drop_enc = ohe_sector.transform(emicron[['GRUPOS4']]).toarray()
drop_enc_df = pd.DataFrame(drop_enc,columns = ohe_sector.get_feature_names_out())
drop_enc_df.drop('GRUPOS4_2', axis=1, inplace = True) # Utilizar como categoría de referencia 2: Industria Manufacturera

emicron = emicron.join(drop_enc_df)

ohe_conta = OneHotEncoder().fit(emicron[['REG_CONT']])
drop_enc = ohe_conta.transform(emicron[['REG_CONT']]).toarray()
drop_enc_df = pd.DataFrame(drop_enc,columns = ohe_conta.get_feature_names_out())
drop_enc_df.drop('REG_CONT_4', axis=1, inplace = True) # Utilizar como categoría de referencia 4: No lleva registros (aplica sólo para Establecimientos)

emicron = emicron.join(drop_enc_df)

# ohe_sector12 = OneHotEncoder().fit(emicron[['GRUPOS12']])
# drop_enc = ohe_sector12.transform(emicron[['GRUPOS12']]).toarray()
# drop_enc_df = pd.DataFrame(drop_enc,columns = ohe_sector12.get_feature_names_out())
# drop_enc_df.drop('GRUPOS12_4', axis=1, inplace = True)

# emicron = emicron.join(drop_enc_df)

ohe_anoe = OneHotEncoder().fit(emicron[['ANOS_ESPACIO']])
drop_enc = ohe_anoe.transform(emicron[['ANOS_ESPACIO']]).toarray()
drop_enc_df = pd.DataFrame(drop_enc,columns = ohe_anoe.get_feature_names_out())
drop_enc_df.drop('ANOS_ESPACIO_1', axis=1, inplace = True) # Utilizar como categoría de referencia 1: Menos de un año

emicron = emicron.join(drop_enc_df)

### 6.2 Eliminar variables base

In [ ]:
## Eliminar las variables de las que se partió para la dicotomización
# Código aplicable para la base de establecimientos
emicron.drop(['COD_DEPTO','UBICA','GRUPOS4','REG_CONT'],axis=1, inplace= True)  
# Código aplicable para la base de puestos móviles
emicron.drop(['COD_DEPTO','GRUPOS4','ANOS_ESPACIO'],axis=1, inplace= True)